In [5]:
!pip3 install requests
!pip3 install bs4
!pip install pymongo
!pip install fake_useragent

You should consider upgrading via the 'C:\Users\Simran Shah\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Simran Shah\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Simran Shah\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Simran Shah\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [3]:
client = MongoClient('')

In [4]:
db = client['BeyondPrice']
products = db['Products']

In [7]:
# laptops_list_hp_amazon = [
# 'https://www.amazon.com/dp/B01J24H2K0/ref=vp_d_ac_sub_sp_pd?_encoding=UTF8&pf_rd_p=d7[…]4991d6f7038&pd_rd_r=7e5e62b0-0e9a-42cf-a358-ae0a49b91f5e&th=1'
# ]

In [8]:
# laptops_list_hp_ebay = [
# 'https://www.ebay.com/itm/334756920660?hash=item4df1113d54:g:4boAAOSw0vZj89Yv&amdata=enc%3AAQAHAAAA8GZpbpDqvcYP8NxADxh%2FRTW9CGV93QWwvzCFiyGYV%2FyWceojqF72qBwU3iYPv0OyBRp5IxkwP%2BY3AZM5R32zQMqz8wGHM8kQeifWqHttDYojePgC1lzi9YuuThAbX7%2BgEV0f%2Bjneh4aQ2n2ny0FcrMH6qpAqfgEAQ0o82nt3SR2CYklt8uwDpqbW97ZoGBDE5nu%2FcvC9RRWaDDUqvr0NbTr5b2LC8uhjJlK6uwJAqF5OapdIdK20gYwMEbFEsmGO9aV3OrZFbOn5tya539A4SXR7caLi6HWPvZ4THXLrzKkE1oaFAqZaGs6t41A18oWwJQ%3D%3D%7Ctkp%3ABk9SR-6ehZ3SYQ'
# ]

In [9]:
from time import sleep
import urllib.request
from fake_useragent import UserAgent

def web_scrape_info(url_list_amazon, url_list_ebay):
    product_dict = {}
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    ua=UserAgent()
    hdr = {'User-Agent': ua.random,
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}
    
    for i in range(len(url_list_amazon)):
    
        html = requests.get(url_list_amazon[i], headers=hdr)
        soup = BeautifulSoup(html.content, 'html.parser')

        #Id
        product_dict['_id'] = "Dell"+str(1)

        #Product ID
        product_id = soup.find('input', {'id':"ASIN"})['value']
        print('Product ID: ',product_id)
        product_dict['Product_ID'] = product_id 

        #Category
        product_dict['Category'] = 'Laptops'

        #Category ID
        product_dict['Category ID'] = 'C04'

        #Brand
        product_dict['Brand'] = 'Dell'

        #Product Name
        product_name = soup.find('span', {'id' : "productTitle"}).text.strip()
        print(product_name)
        product_dict['Product_Name'] = product_name

        #Price
        price = soup.find_all('span', class_ ="a-offscreen")[0]
        print('Price: ', price.text)
        product_dict['Price'] = price.text

        #Image URL
        image_url = soup.find('div', {'id' : "imgTagWrapperId"})
        print('Image URL', image_url.find('img')['src'])
        product_dict['Image URL'] = image_url.find('img')['src']

        #Amazon URL
        product_dict['Amazon_URL'] = url_list_amazon[i]

        #Ebay URL
        product_dict['Ebay_URL'] = url_list_ebay[i]

        #Reviews (getting url of all the reviews "See All")
        see_all_reviews_url = soup.find('div', {'id' : "reviews-medley-footer"})
        url_reviews = 'https://www.amazon.com'+see_all_reviews_url.find('a')['href']
        print('Reviews URL: ',url_reviews)

        html_reviews = requests.get(url_reviews, headers=hdr)
        soup_reviews = BeautifulSoup(html_reviews.text)

        #Reviews
        review_info = soup_reviews.find("div", {"data-hook": "cr-filter-info-review-rating-count"}).text.strip()
        total_no_reviews = int(review_info.split('total ratings,')[1].strip().split(' ')[0].replace(',',''))
        print('Total No. of reviews: ',total_no_reviews)
        reviews = []
        for review in soup_reviews.find_all("span", {"data-hook": "review-body"}):
            reviews.append(review.get_text().replace('\n',''))

        #For pagination of reviews
        n=2
        if total_no_reviews > 10:
            mid_url=soup_reviews.find("div", {"id" : "cm_cr-pagination_bar"}).find('a')['href']
            print("MID URL: ",mid_url)
            next_url = 'https://www.amazon.com'+mid_url
            while n<=4:
                try:
                    html_next = requests.get(next_url, headers=hdr)
                    soup_next = BeautifulSoup(html_next.text)
                    reviews_count_page = soup_next.find_all("span", {"data-hook": "review-body"})
                    for review in reviews_count_page:
                        reviews.append(review.get_text().replace('\n',''))
                    if len(reviews_count_page) == 10:
                        #if (total_no_reviews % 10 != 0):
                        next_url = 'https://www.amazon.com'+soup_next.find("div", {"id" : "cm_cr-pagination_bar"}).find('li', class_ ="a-last").find('a')['href']
                    n+=1
                except requests.exceptions.ConnectionError as e:
                    print(e)
            print('Size of list reviews: ',len(reviews))
            product_dict['Reviews'] = reviews
            product_id = products.insert_one(product_dict)

    return 

In [10]:
web_scrape_info(laptops_list_hp_amazon, laptops_list_hp_ebay)

Product ID:  B01J24H2K0
Amazon Basics Carry-On Travel Backpack - Black
Price:  $49.49


In [6]:
db.list_collection_names()

['Credentials', 'Products']

In [100]:
s = db.Products.find({"Category":"Mobiles"})

In [101]:
from statistics import mean 
price = []
for i in s:
    price.append(float(i['Price'][1:].replace(',','')))
print(price)
avg = round(mean(price),2)
print('Min price:',min(price))
print('Max price:',max(price))
print('Avg price:',avg)

count_before_avg = 0
count_after_avg = 0
for p in price:
    if p <= avg:
        count_before_avg += 1
    else:
        count_after_avg += 1
print('Before avg: ', count_before_avg)
print('After avg: ', count_after_avg)

[740.0, 822.26, 529.99, 194.99, 639.0, 364.99, 419.0, 84.0, 299.0, 749.0, 49.99, 148.72, 249.99, 169.99, 249.99, 82.99]
Min price: 49.99
Max price: 822.26
Avg price: 362.12
Before avg:  9
After avg:  7


In [10]:
db.products.update_many({'Category ID': 'C01'},
                       {'$set' : {'Category' : 'Shoes'}        
})

myquery = {'Category ID': 'C01'}

newvalues = {'$set' : {'Category' : 'Shoes'}}

x = mycol.update_many(myquery, newvalues)

print(x.modified_count, "documents updated.")


In [12]:
from pymongo import MongoClient
client = MongoClient('')

db = client['BeyondPrice']
products = db['Products']

list_products = []
product_category = products.find({"Category":"Mobiles"})

for prod in product_category:
    prod_price = float(prod['Price'][1:].replace(',',''))
    if 45.00 <= prod_price <= 362.12:
        prod_spec = {}
        prod_spec['Brand'] = prod['Brand']
        prod_spec['Product_Name'] = prod['Product_Name']
        prod_spec['price'] = prod['Price']
        prod_spec['Image_URL'] = prod['Image URL']
        prod_spec['Sentiment Score'] = prod['sentiment_score_vd']
        list_products.append(prod_spec)
#print(list_products)
#print('len',len(list_products))
newlist = sorted(list_products, key=lambda d: d['Sentiment Score'], reverse=True) 
print('Sorted list: ', newlist[0:5])

Sorted list:  [{'Brand': 'TCL', 'Product_Name': 'TCL 20 Pro 5G Unlocked Smartphone with 6.67” AMOLED FHD+ Display, 48MP OIS Quad Camera, 6GB+256GB, 4500mAh Battery, US 5G Verizon Cellphone, Marine Blue (Does not Support AT&T 5G)', 'price': '$249.99', 'Image_URL': 'https://m.media-amazon.com/images/I/81-iFZ14NGS.__AC_SX300_SY300_QL70_ML2_.jpg', 'Sentiment Score': 0.5978852348993287}, {'Brand': 'Google', 'Product_Name': 'Google Pixel 6a - 5G Android Phone - Unlocked Smartphone with 12 Megapixel Camera and 24-Hour Battery - Sage', 'price': '$299.00', 'Image_URL': 'https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/61dNAU6PgxL.__AC_SX300_SY300_QL70_ML2_.jpg', 'Sentiment Score': 0.5569153333333335}, {'Brand': 'OnePlus', 'Product_Name': 'OnePlus Nord N20 5G |Android Smart Phone |6.43" AMOLED Display|6+128GB |U.S. Unlocked |4500 mAh Battery | 33W Fast Charging | Blue Smoke', 'price': '$249.99', 'Image_URL': 'https://m.media-amazon.com/images/I/71hIjJkMqFL.__AC_SX300_SY300_Q

In [39]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
def web_scrape_ebay(ebay_url):

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    ua=UserAgent()
    hdr = {'User-Agent': ua.random,
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}
    
    html = requests.get(ebay_url, headers=hdr)
    soup = BeautifulSoup(html.content, 'html.parser')

    #Price
    msg = soup.find('span', class_ ="ux-textspans ux-textspans--BOLD")
    print(msg.text)
    if msg.text == 'Seller information':
        price_eb = soup.find('span', {'itemprop':"price"}).text.strip()
        print(price_eb[3:])
    else:
        price_eb = 'The item is no longer available'
        print(price_eb)
        
        

In [40]:
t = web_scrape_ebay('https://www.ebay.com/itm/255356292175?hash=item3b746bcc4f%3Ag%3AlIMAAOSwfcZh86Xr&am[…]NJIAaM1HEGG0E8Rz2%7Ctkp%3ABk9SR77alrHTYQ&LH_ItemCondition=1000')

Seller information
$93.80
